<a href="https://colab.research.google.com/github/toniskai/BH/blob/master/Predicci%C3%B3n%20de%20Demanda%20con%20Prophet%20ALE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
import itertools
import matplotlib.pyplot as plt

# 1. Cargar y preparar los datos
def cargar_datos():
    df = pd.read_excel('/content/Demanda ALE v2.xlsx', sheet_name='Hoja1')
    df['FSol_Eje'] = pd.to_datetime(df['FSol_Eje'])

    # Limpieza y transformación
    df = df.rename(columns={
        'NumSemana': 'num_semana',
        'DiaSemana': 'dia_semana',
        'festivoNacional': 'festivo_nacional',
        'festivoLocal': 'festivo_local',
        'finDeSemana': 'fin_de_semana',
        'NumSolicitudes': 'solicitudes'
    })

    df['dia_semana'] = df['dia_semana'].map({
        'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3,
        'viernes': 4, 'sábado': 5, 'domingo': 6
    })

    prophet_df = df[['FSol_Eje', 'solicitudes']].copy()
    prophet_df.columns = ['ds', 'y']

    regresores = ['num_semana', 'dia_semana', 'festivo_nacional', 'festivo_local', 'fin_de_semana', 'Verano']
    for reg in regresores:
        prophet_df[reg] = df[reg]

    return prophet_df, df

# 2. Función para ajuste de hiperparámetros
def ajustar_hiperparametros(df):
    param_grid = {
        'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.5],
        'seasonality_prior_scale': [0.1, 1.0, 10.0],
        'holidays_prior_scale': [0.1, 1.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative']
    }

    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    mapes = []

    for params in all_params:
        model = Prophet(**params,
                       yearly_seasonality=True,
                       weekly_seasonality=True,
                       daily_seasonality=False)

        # Añadir regresores
        regresores = ['num_semana', 'dia_semana', 'festivo_nacional', 'festivo_local', 'fin_de_semana', 'Verano']
        for reg in regresores:
            model.add_regressor(reg)

        model.fit(df)

        # Validación cruzada
        # Increased horizon to allow for MAPE calculation
        df_cv = cross_validation(model, initial='730 days', period='180 days', horizon='365 days', parallel="processes")
        df_p = performance_metrics(df_cv)

        # Extract MAPE directly from the 'mape' column
        # Check if 'mape' is in the columns before accessing it
        if 'mape' in df_p.columns:
            mapes.append(df_p['mape'].mean())
        else:
            print("Warning: 'mape' not found in performance metrics. Skipping this parameter set.")
            mapes.append(float('inf'))  # Assign a high value to avoid selecting this parameter set

    tuning_results = pd.DataFrame(all_params)
    tuning_results['mape'] = mapes

    best_params = all_params[mapes.index(min(mapes))]
    print(f"Mejores parámetros encontrados: {best_params}")
    print(f"MAPE mínimo: {min(mapes)}")

    return best_params

# 3. Entrenar modelo con los mejores parámetros
def entrenar_modelo(df, best_params):
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        **best_params
    )

    # Añadir regresores
    regresores = ['num_semana', 'dia_semana', 'festivo_nacional', 'festivo_local', 'fin_de_semana', 'Verano']
    for reg in regresores:
        model.add_regressor(reg)

    model.fit(df)
    return model

# 4. Validación cruzada y evaluación
def validar_modelo(model, df):
    # Adjusted cross-validation parameters
    # Increased initial to provide enough data for the first cutoff
    # Making sure that initial + period + horizon <= total data length
    initial_days = '365 days'  # Reduced to 365 days
    period_days = '90 days'
    horizon_days = '30 days'

    total_data_length = len(df)
    initial = pd.to_timedelta(initial_days).days
    period = pd.to_timedelta(period_days).days
    horizon = pd.to_timedelta(horizon_days).days

    # Adjust parameters if needed to avoid issues
    if initial + period + horizon > total_data_length:
        # Calculate maximum possible horizon
        max_horizon = total_data_length - (initial + period)
        if max_horizon <= 0:
            raise ValueError("Not enough data for cross-validation with the given initial and period.")

        horizon_days = f"{max_horizon} days"
        horizon = max_horizon
        print(f"Warning: Adjusted horizon to {horizon_days} to ensure enough data for cross-validation.")

    # ***DIAGNOSE EMPTY df_cv***
    # Check if cross_validation would return an empty DataFrame
    # before actually calling it
    cutoffs = pd.Series(pd.to_datetime(df['ds'].unique())).sort_values().to_numpy()

    # Filter cutoffs to ensure enough data for initial, period and horizon
    cutoffs = cutoffs[cutoffs + pd.Timedelta(days=initial + period + horizon) <= df['ds'].max()]

    if len(cutoffs) == 0:
        print("Warning: Not enough data for cross-validation. Skipping cross-validation and plotting.")
        return pd.DataFrame() # Return an empty DataFrame to avoid further errors
    # ***END DIAGNOSE EMPTY df_cv***

    df_cv = cross_validation(
        model,
        initial=initial_days,
        period=period_days,
        horizon=horizon_days,
        parallel="processes"
    )

    # Check if df_cv is empty to avoid error in performance_metrics
    if df_cv.empty:
        print("Warning: Cross-validation DataFrame is empty. Skipping performance metrics and plotting.")
        return pd.DataFrame()

    df_p = performance_metrics(df_cv)
    print("\nMétricas de validación cruzada:")
    print(df_p.head())

    # Check if df_cv is empty before plotting
    if df_cv.empty:
        print("Warning: Cross-validation DataFrame is empty. Skipping plotting.")
    else:
        # Check if 'horizon' column exists and has timedelta values before plotting
        if 'horizon' in df_cv.columns and pd.api.types.is_timedelta64_dtype(df_cv['horizon']):
            fig = plot_cross_validation_metric(df_cv, metric='mape')
            plt.title('Error Porcentual Absoluto Medio (MAPE) en Validación Cruzada')
            plt.show()
        else:
            print("Warning: 'horizon' column not found or not in timedelta format. Skipping plotting.")

    return df_p

# 5. Preparar datos futuros y hacer predicciones
def hacer_predicciones(model, original_df):
    future_dates = pd.date_range(start='2025-03-01', end='2025-03-31')
    future = pd.DataFrame({'ds': future_dates})

    # Calcular regresores para fechas futuras
    future['num_semana'] = future['ds'].dt.isocalendar().week
    future['dia_semana'] = future['ds'].dt.weekday
    future['festivo_nacional'] = 0  # Ajustar según calendario 2025
    future['festivo_local'] = 0     # Ajustar según calendario 2025
    future['fin_de_semana'] = (future['ds'].dt.weekday >= 5).astype(int)
    future['Verano'] = (future['ds'].dt.month.isin([6,7,8])).astype(int)

    # Ensure all regressor columns are of numeric type, specifically int64 or float64
    for col in ['num_semana', 'dia_semana', 'festivo_nacional', 'festivo_local', 'fin_de_semana', 'Verano']:
        future[col] = pd.to_numeric(future[col]).astype(int) # or .astype(float) if needed

    forecast = model.predict(future)

    # Seleccionar columnas relevantes
    resultados = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    resultados.columns = ['Fecha', 'Predicción', 'Límite Inferior', 'Límite Superior']
    resultados['Fecha'] = resultados['Fecha'].dt.strftime('%Y-%m-%d')

    return resultados

# 6. Guardar resultados en Excel
def guardar_resultados(resultados, metricas, best_params, filename='prediccion_demanda_ALE.xlsx'):
    with pd.ExcelWriter(filename) as writer:
        resultados.to_excel(writer, sheet_name='Predicciones', index=False)
        metricas.to_excel(writer, sheet_name='Métricas Validación', index=False)
        pd.DataFrame([best_params]).to_excel(writer, sheet_name='Mejores Parámetros', index=False)

    print(f"\nResultados guardados en {filename}")

# Ejecución principal
def main():
    # Cargar datos
    prophet_df, original_df = cargar_datos()

    # Ajuste de hiperparámetros (puede tomar tiempo)
    print("Ajustando hiperparámetros...")
    best_params = ajustar_hiperparametros(prophet_df)

    # Entrenar modelo con mejores parámetros
    print("\nEntrenando modelo con los mejores parámetros...")
    model = entrenar_modelo(prophet_df, best_params)

    # Validar modelo
    print("\nValidando modelo...")
    metricas = validar_modelo(model, prophet_df)

    # Hacer predicciones
    print("\nGenerando predicciones...")
    resultados = hacer_predicciones(model, original_df)
    print("\nPredicción de demanda para Marzo 2025:")
    print(resultados.to_string(index=False))

    # Guardar resultados
    guardar_resultados(resultados, metricas, best_params)

if __name__ == "__main__":
    main()

DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/uzqaapwf.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/l2902hvz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=27293', 'data', 'file=/tmp/tmpmupks3c6/uzqaapwf.json', 'init=/tmp/tmpmupks3c6/l2902hvz.json', 'output', 'file=/tmp/tmpmupks3c6/prophet_modelckzf60xe/prophet_model-20250404213740.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:37:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


Ajustando hiperparámetros...


21:37:40 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 3 forecasts with cutoffs between 2023-03-13 00:00:00 and 2024-03-07 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7eea5fe7b690>
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/dmjh5zds.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/l_9wqlp2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/fyrc3fg8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86034', 'data', 'file=/tmp/tmpmupks3c6/dmjh5zds.json', 'init=/tmp/tmpmupks3c6/fyrc3fg8.json', 'output', 'file=/tmp/tmpmupks3c6/prophet_modelbl0c64ay/prophet_model-20250404213740.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:37:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdsta

21:37:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 3 forecasts with cutoffs between 2023-03-13 00:00:00 and 2024-03-07 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7eea5f1f27d0>
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/_eb2vqll.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/0rj2x4wx.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/0msnroxg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=37363', 'data', 'file=/tmp/tmpmupks3c6/_eb2vqll.json', 'init=/tmp/tmpmupks3c6/0msnroxg.json', 'output', 'file=/tmp/tmpmupks3c6/prophet_modelk7rg13v9/prophet_model-20250404213742.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:37:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdsta

DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/rjg2elxz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=53187', 'data', 'file=/tmp/tmpmupks3c6/_1iqlu1_.json', 'init=/tmp/tmpmupks3c6/rjg2elxz.json', 'output', 'file=/tmp/tmpmupks3c6/prophet_modelc4f3xyf8/prophet_model-20250404213743.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:37:43 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:37:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 3 forecasts with cutoffs between 2023-03-13 00:00:00 and 2024-03-07 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7eea5f497910>
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/n4gvwlh2.json
DEBUG:cmdstanpy:input tempfile: 

...

Mejores parámetros encontrados: {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.1, 'holidays_prior_scale': 0.1, 'seasonality_mode': 'additive'}
MAPE mínimo: inf

Entrenando modelo con los mejores parámetros...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/2hfilh0u.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=75617', 'data', 'file=/tmp/tmpmupks3c6/qb5pr5f7.json', 'init=/tmp/tmpmupks3c6/2hfilh0u.json', 'output', 'file=/tmp/tmpmupks3c6/prophet_modelfpmzygo5/prophet_model-20250404214018.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:40:18 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:40:18 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 13 forecasts with cutoffs between 2022-02-21 00:00:00 and 2025-02-05 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7eea5f7f4d90>



Validando modelo...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/ui3k0jdc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/y87hswyb.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/cp7mr84g.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=71542', 'data', 'file=/tmp/tmpmupks3c6/ui3k0jdc.json', 'init=/tmp/tmpmupks3c6/cp7mr84g.json', 'output', 'file=/tmp/tmpmupks3c6/prophet_modelni2rkky9/prophet_model-20250404214018.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:40:18 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmupks3c6/kfdp1etc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=31890', 'data


Métricas de validación cruzada:
  horizon        mse      rmse       mae     mdape     smape  coverage
0  3 days  11.093514  3.330693  2.291354  0.380305  0.949829  0.846154
1  4 days  17.663643  4.202814  2.540380  0.338463  0.866958  0.820513
2  5 days  18.919550  4.349661  2.556134  0.301422  0.798033  0.820513
3  6 days  20.544689  4.532625  2.819623  0.466262  0.879896  0.820513
4  7 days  12.948845  3.598450  2.482460  0.562325  0.985641  0.846154

Generando predicciones...

Predicción de demanda para Marzo 2025:
     Fecha  Predicción  Límite Inferior  Límite Superior
2025-03-01    0.732202        -4.044384         5.497887
2025-03-02    0.758076        -4.196748         5.578886
2025-03-03    8.624898         3.804325        13.449080
2025-03-04    9.306401         4.746511        14.173039
2025-03-05    7.773870         2.830788        12.667174
2025-03-06    7.607603         2.883709        12.466449
2025-03-07    6.871882         1.979994        11.484435
2025-03-08    1.21